# 基于MindSpore实现AlexNet手写字体识别

## 下载MNIST数据集

In [1]:
import mindspore
from mindspore import nn
from mindspore import ops
from mindspore.dataset import vision, transforms
from mindspore.dataset import MnistDataset


# Download data from open datasets
from download import download

url = "https://mindspore-website.obs.cn-north-4.myhuaweicloud.com/" \
      "notebook/datasets/MNIST_Data.zip"
path = download(url, "./", kind="zip", replace=True)


file_sizes: 100%|██████████████████████████| 10.8M/10.8M [00:00<00:00, 46.9MB/s]
Extracting zip file...
Successfully downloaded / unzipped to ./


In [2]:
train_dataset = MnistDataset('MNIST_Data/train')
test_dataset = MnistDataset('MNIST_Data/test')

train_dataset.get_col_names()

['image', 'label']

In [3]:

def datapipe(dataset, batch_size):
    image_transforms = [
        vision.Rescale(1.0 / 255.0, 0),
        vision.Resize((32, 32)),
        vision.HWC2CHW()
        
    ]
    label_transform = transforms.TypeCast(mindspore.int32)

    dataset = dataset.map(image_transforms, 'image')
    dataset = dataset.map(label_transform, 'label')
    dataset = dataset.batch(batch_size)
    return dataset


# Map vision transforms and batch dataset
train_dataset = datapipe(train_dataset, 64)
test_dataset = datapipe(test_dataset, 64)



In [4]:

for image, label in test_dataset.create_tuple_iterator():
    print(f"Shape of image [N, C, H, W]: {image.shape} {image.dtype}")
    print(f"Shape of label: {label.shape} {label.dtype}")
    break

for data in test_dataset.create_dict_iterator():
    print(f"Shape of image [N, C, H, W]: {data['image'].shape} {data['image'].dtype}")
    print(f"Shape of label: {data['label'].shape} {data['label'].dtype}")
    break


Shape of image [N, C, H, W]: (64, 1, 32, 32) Float32
Shape of label: (64,) Int32
Shape of image [N, C, H, W]: (64, 1, 32, 32) Float32
Shape of label: (64,) Int32


## 定义一个减少卷积层的精简版AlexNet

由于受到计算机性能的影响，虽然LeNet在图像分类中取得了较好的成绩，但是并没有引起很多的关注。 知道2012年，Alex等人提出的AlexNet网络在ImageNet大赛上以远超第二名的成绩夺冠，卷积神经网络乃至深度学习重新引起了广泛的关注。AlexNet是在LeNet的基础上加深了网络的结构，学习更丰富更高维的图像特征。

In [19]:
'''
AlexNet的原始输入图片大小为224*224
Mnist数据集中图片大小为28*28
所以需要对网络进行精简，减少两层卷积层。
'''

import mindspore.nn as nn



import mindspore.nn as nn

class AlexNet(nn.Cell):
    def __init__(self, num_class=10, num_channel=1):
        super(AlexNet, self).__init__()
        # 卷积层，输入的通道数为num_channel，输出的通道数为6，卷积核大小为5*5
        self.conv1 = nn.Conv2d(num_channel, 6, 5, pad_mode='valid')
        # 卷积层，输入的通道数为6，输出的通道数为16，卷积核大小为5*5
        self.conv2 = nn.Conv2d(6, 16, 5, pad_mode='valid')
        # 全连接层，输入个数为16*5*5，输出个数为120
        self.fc1 = nn.Dense(16 * 5 * 5, 120)
        # 全连接层，输入个数为120，输出个数为84
        self.fc2 = nn.Dense(120, 84)
        # 全连接层，输入个数为84，分类的个数为num_class
        self.fc3 = nn.Dense(84, num_class)
        # ReLU激活函数
        self.relu = nn.ReLU()
        # 池化层
        self.max_pool2d = nn.MaxPool2d(kernel_size=2, stride=2)
        # 多维数组展平为一维数组
        self.flatten = nn.Flatten()

    def construct(self, x):
        # 使用定义好的运算构建前向网络
        x = self.conv1(x)
        x = self.relu(x)
        x = self.max_pool2d(x)
        x = self.conv2(x)
        x = self.relu(x)
        x = self.max_pool2d(x)
        x = self.flatten(x)
        x = self.fc1(x)
        x = self.relu(x)
        x = self.fc2(x)
        x = self.relu(x)
        x = self.fc3(x)
        return x

model = AlexNet()


## 损失函数、优化器

In [20]:
# Instantiate loss function and optimizer
loss_fn = nn.CrossEntropyLoss()
optimizer = nn.Adam(params=model.trainable_params())

[Parameter (name=conv1.weight, shape=(6, 1, 5, 5), dtype=Float32, requires_grad=True), Parameter (name=conv2.weight, shape=(16, 6, 5, 5), dtype=Float32, requires_grad=True), Parameter (name=fc1.weight, shape=(120, 400), dtype=Float32, requires_grad=True), Parameter (name=fc1.bias, shape=(120,), dtype=Float32, requires_grad=True), Parameter (name=fc2.weight, shape=(84, 120), dtype=Float32, requires_grad=True), Parameter (name=fc2.bias, shape=(84,), dtype=Float32, requires_grad=True), Parameter (name=fc3.weight, shape=(10, 84), dtype=Float32, requires_grad=True), Parameter (name=fc3.bias, shape=(10,), dtype=Float32, requires_grad=True)]


## 训练模型

In [21]:
def train(model, dataset, loss_fn, optimizer):
    # Define forward function
    def forward_fn(data, label):
        logits = model(data)
        loss = loss_fn(logits, label)
        return loss, logits

    # Get gradient function
    grad_fn = ops.value_and_grad(forward_fn, None, optimizer.parameters, has_aux=True)

    # Define function of one-step training
    def train_step(data, label):
        (loss, _), grads = grad_fn(data, label)
        loss = ops.depend(loss, optimizer(grads))
        return loss

    size = dataset.get_dataset_size()
    model.set_train()
    for batch, (data, label) in enumerate(dataset.create_tuple_iterator()):
        loss = train_step(data, label)

        if batch % 100 == 0:
            loss, current = loss.asnumpy(), batch
            print(f"loss: {loss:>7f}  [{current:>3d}/{size:>3d}]")


def test(model, dataset, loss_fn):
    num_batches = dataset.get_dataset_size()
    model.set_train(False)
    total, test_loss, correct = 0, 0, 0
    for data, label in dataset.create_tuple_iterator():
        pred = model(data)
        total += len(data)
        test_loss += loss_fn(pred, label).asnumpy()
        correct += (pred.argmax(1) == label).asnumpy().sum()
    test_loss /= num_batches
    correct /= total
    print(f"Test: \n Accuracy: {(100*correct):>0.1f}%, Avg loss: {test_loss:>8f} \n")

In [27]:

epochs = 3
for t in range(epochs):
    print(f"Epoch {t+1}\n-------------------------------")
    train(model, train_dataset, loss_fn, optimizer)
    test(model, test_dataset, loss_fn)
print("Done!")


Epoch 1
-------------------------------


[WARNING] DEVICE(80609,ffffbc6d6780,python):2022-11-27-23:21:01.790.549 [mindspore/ccsrc/plugin/device/ascend/hal/device/kernel_select_ascend.cc:330] FilterRaisedOrReducePrecisionMatchedKernelInfo] Operator:[Default/flatten-Flatten/GatherD-op1882] don't support int64, reduce precision from int64 to int32.


loss: 0.040397  [  0/938]
loss: 0.035882  [100/938]
loss: 0.011926  [200/938]
loss: 0.031093  [300/938]
loss: 0.148474  [400/938]
loss: 0.050996  [500/938]
loss: 0.060591  [600/938]
loss: 0.104810  [700/938]
loss: 0.101236  [800/938]
loss: 0.020467  [900/938]


[WARNING] DEVICE(80609,ffffbc6d6780,python):2022-11-27-23:21:20.297.717 [mindspore/ccsrc/plugin/device/ascend/hal/device/kernel_select_ascend.cc:330] FilterRaisedOrReducePrecisionMatchedKernelInfo] Operator:[Default/flatten-Flatten/GatherD-op2152] don't support int64, reduce precision from int64 to int32.


Test: 
 Accuracy: 98.0%, Avg loss: 0.059372 

Epoch 2
-------------------------------


...

Test: 
 Accuracy: 98.0%, Avg loss: 0.059503 

Epoch 3
-------------------------------


[WARNING] DEVICE(80609,ffffbc6d6780,python):2022-11-27-23:21:53.270.497 [mindspore/ccsrc/plugin/device/ascend/hal/device/kernel_select_ascend.cc:330] FilterRaisedOrReducePrecisionMatchedKernelInfo] Operator:[Default/flatten-Flatten/GatherD-op2848] don't support int64, reduce precision from int64 to int32.


loss: 0.067681  [  0/938]
loss: 0.004182  [100/938]
loss: 0.037411  [200/938]
loss: 0.025833  [300/938]
loss: 0.068297  [400/938]
loss: 0.110484  [500/938]
loss: 0.065561  [600/938]
loss: 0.015396  [700/938]
loss: 0.023539  [800/938]
loss: 0.019014  [900/938]


[WARNING] DEVICE(80609,ffffbc6d6780,python):2022-11-27-23:22:07.939.315 [mindspore/ccsrc/plugin/device/ascend/hal/device/kernel_select_ascend.cc:330] FilterRaisedOrReducePrecisionMatchedKernelInfo] Operator:[Default/flatten-Flatten/GatherD-op3061] don't support int64, reduce precision from int64 to int32.


Test: 
 Accuracy: 98.0%, Avg loss: 0.060285 

Done!



## 保存模型

In [23]:

# Save checkpoint
mindspore.save_checkpoint(model, "model.ckpt")
print("Saved Model to model.ckpt")

Saved Model to model.ckpt


## 加载模型

In [25]:

# Instantiate a random initialized model
model = AlexNet()
# Load checkpoint and load parameter to model
param_dict = mindspore.load_checkpoint("model.ckpt")
param_not_load = mindspore.load_param_into_net(model, param_dict)
print(param_not_load)


[]


## 预测推理

In [26]:

model.set_train(False)
for data, label in test_dataset:
    pred = model(data)
    predicted = pred.argmax(1)
    print(f'Predicted: "{predicted[:10]}", Actual: "{label[:10]}"')
    break



Predicted: "[3 2 6 1 8 9 9 3 8 9]", Actual: "[3 2 6 1 8 9 9 3 8 9]"
